In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-291905
Azure region: westeurope
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-291905


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "myCluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    my_compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except:
    my_compute_cluster_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    my_compute_cluster = ComputeTarget.create(ws, cluster_name, my_compute_cluster_config)

my_compute_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [2]:
pip install azureml-defaults scikit-learn

  Preparing metadata (setup.py) ... - \ done
INFO: pip is looking at multiple versions of azure-monitor-opentelemetry-exporter to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 136.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.7 MB/s eta 0:00:00
  DEPRECATION: Building 'fusepy' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'fusepy'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created w

In [5]:
pip install azureml-widgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.7 MB/s  0:00:00
  Attempting uninstall: azureml-core
    Found existing installation: azureml-core 1.61.0
    Uninstalling azureml-core-1.61.0:
      Successfully uninstalled azureml-core-1.61.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-defaults 1.61.0 requires azureml-core~=1.61.0, but you have azureml-core 1.60.0.post1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install azureml-train

  Using cached azureml_core-1.61.0-py3-none-any.whl.metadata (3.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 58.8 MB/s  0:00:00
Using cached azureml_core-1.61.0-py3-none-any.whl (3.3 MB)
  Attempting uninstall: azureml-core
    Found existing installation: azureml-core 1.60.0.post1
    Uninstalling azureml-core-1.60.0.post1:
      Successfully uninstalled azureml-core-1.60.0.post1━━━━━━━━━━ 1/5 [azureml-core]
  Attempting uninstall: azureml-telemetry━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [azureml-core]
    Found existing installation: azureml-telemetry 1.60.0━━━━━ 1/5 [azureml-core]
    Uninstalling azureml-telemetry-1.60.0:━━━━━━━━━━━━━━━━━━━━ 1/5 [azureml-core]
      Successfully uninstalled azureml-telemetry-1.60.0━━━━━━━ 1/5 [azureml-core]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [azureml-train]0m [azureml-train-core]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following

In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.1,1),
        '--max_iter': choice(10,20,50)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=cluster_name,
    environment=sklearn_env    
    )

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     run_config=src,
                                     max_total_runs=4,
                                     max_concurrent_runs=4)

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ede07e77-be40-4455-88d1-10df18df9cf0
Web View: https://ml.azure.com/runs/HD_ede07e77-be40-4455-88d1-10df18df9cf0?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-291905/workspaces/quick-starts-ws-291905&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-12-05T09:16:03.3974782Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-12-05T09:16:03.9142126Z][SCHEDULER][INFO]Scheduling job, id='HD_ede07e77-be40-4455-88d1-10df18df9cf0_0' 
[2025-12-05T09:16:03.9776510Z][SCHEDULER][INFO]Scheduling job, id='HD_ede07e77-be40-4455-88d1-10df18df9cf0_1' 
[2025-12-05T09:16:04.0797698Z][SCHEDULER][INFO]Scheduling job, id='HD_ede07e77-be40-4455-88d1-10df18df9cf0_3' 
[2025-12-05T09:16:04.0807090Z][SCHEDULER][INFO]Scheduling job, id='HD_ede07e77-be40-4455-88d1-10df18df9cf0_2' 
[2025-12-05T09:16:05.2045209Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_ede07e77-be40-4455-88d1-10df18df9cf0_0' 
[2025-12-05T09:1

In [10]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics() 
best_run_file_names = best_run.get_file_names()

print('Best Run Id: ', best_run.id) 
print('Best Run Metrics: ', best_run_metrics) 
print('Best Run Accuracy:', best_run_metrics['Accuracy']) 
print('Best Run File Names: ', best_run_file_names) 

Best Run Id:  HD_ede07e77-be40-4455-88d1-10df18df9cf0_0
Best Run Metrics:  {'Max iterations:': 20, 'Regularization Strength:': 1.0, 'Accuracy': 0.8878686327077748}
Best Run Accuracy: 0.8878686327077748
Best Run File Names:  ['azureml-logs/20_image_build_log.txt', 'logs/azureml/dataprep/0/rslex.log.2025-12-05-09', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [5]:
from azureml.core import Workspace, Dataset

subscription_id = '81cefad3-d2c9-4f77-a466-99a7f541c7bb'
resource_group = 'aml-quickstarts-291905'
workspace_name = 'quick-starts-ws-291905'

workspace = Workspace(subscription_id, resource_group, workspace_name)

ds = Dataset.get_by_name(workspace, name='bankmarketing_train')

In [6]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [16]:
pip install azureml-train-automl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 67.7 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 144.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 918.9/918.9 kB 32.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 91.6 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 127.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 121.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.8/934.8 kB 37.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.7 MB/s 

In [7]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)

In [30]:
!pip uninstall ipywidgets -y
!pip install "ipywidgets<8.0.0,>=7.0.0"

Found existing installation: ipywidgets 7.0.0
Uninstalling ipywidgets-7.0.0:
  Successfully uninstalled ipywidgets-7.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.1/124.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.0 MB/s eta 0:00:00:00:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.0.8
    Uninstalling widgetsnbextension-3.0.8:
      Successfully uninstalled widgetsnbextension-3.0.8


In [32]:
!pip install "azureml-core~=1.60.0"
!pip install "azureml-telemetry~=1.60.0"

In [33]:
!pip uninstall psutil -y
!pip install "psutil<5.9.4,>=5.2.2"

Found existing installation: psutil 7.1.3
Uninstalling psutil-7.1.3:
  Successfully uninstalled psutil-7.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.3/292.3 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-nvdashboard 0.13.0 requires jupyterlab>=4, but you have jupyterlab 3.6.8 which is incompatible.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatible.
azureml-training-tabular 1.60.0 requires scipy<1.11.0,>=1.0.0, but you have scipy 1.11.0 which is incompatible.
azureml-training-tabular 1.60.0 requires urllib3<2.0.0, but you have urllib3 2.5.0 which is incompatible.
azureml-automl-runtime 1.60.0 requires urllib3<2.0.0, but you have urllib3 2.5.0 which is incompati

In [8]:
# Submit your automl run
from azureml.core.experiment import Experiment

### YOUR CODE HERE ###
experiment_automl = Experiment(workspace, "automl")

automl_run = experiment_automl.submit(automl_config)
automl_run.wait_for_completion()

/anaconda/envs/azureml_py38/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
2025-12-05 10:13:17.906218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764929599.001803   15244 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764929599.315408   15244 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764929602.118398   15244 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target mo

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_454c115b-11f4-496b-b22a-9141f7135bcd,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2025-12-05:10:13:54,611 INFO     [_base.py:273] Transmission succeeded: Item received: 43. Items accepted: 43
2025-12-05:10:14:09,870 INFO     [_base.py:273] Transmission succeeded: Item received: 6. Items accepted: 6
2025-12-05:10:14:16,244 INFO     [_base.py:273] Transmission succeeded: Item received: 22. Items accepted: 22
2025-12-05:10:14:24,761 INFO     [_base.py:273] Transmission succeeded: Item received: 13. Items accepted: 13
2025-12-05:10:14:47,977 INFO     [_base.py:273] Transmission succeeded: Item received: 5. Items accepted: 5
2025-12-05:10:14:58,301 INFO     [_base.py:273] Transmission succeeded: Item received: 6. Items accepted: 6
2025-12-05:10:15:04,466 INFO     [_base.py:273] Transmission succeeded: Item received: 15. Items accepted: 15
2025-12-05:10:15:25,986 INFO     [_base.py:273] Transmission succeeded: Item received: 9. Items accepted: 9
2025-12-05:10:15:32,3 INFO     [_base.py:273] Transmission succeeded: Item received: 13. Items accepted: 13
2025-12-05:10:15:37,

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run = hyperdrive_run.get_best_run_by_primary_metric()
print("Best run metrics :",best_run_automl.get_metrics())
print("Best run details :",best_run_automl.get_details())
print("Best run file names :",best_run_automl.get_file_names())